In [16]:
import pandas as pd
from numpy import nan
from sklearn.model_selection import train_test_split

# Home Data pipeline

See [readme](../README.md) 2do section for next steps

In [17]:
# load data

global training___data, validation_data, training___price, validation_price
def load_data():
    global training___data, validation_data, training___price, validation_price
    data = pd.read_csv("./train.csv")
    training___data, validation_data, training___price, validation_price = train_test_split(data, data.SalePrice, random_state = 0)

load_data()
training___data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1292,1293,70,RM,60.0,6600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2009,WD,Normal,107500
1018,1019,80,RL,NaN,10784,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,160000
1213,1214,80,RL,NaN,10246,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
1430,1431,60,RL,60.0,21930,Pave,NaN,IR3,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,192140
810,811,20,RL,78.0,10140,Pave,NaN,Reg,Lvl,AllPub,...,648,Fa,GdPrv,NaN,0,1,2006,WD,Normal,181000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,764,60,RL,82.0,9430,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,337000
835,836,20,RL,60.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2010,WD,Normal,128000
1216,1217,90,RM,68.0,8930,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,112000
559,560,120,RL,NaN,3196,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,234000


In [18]:
# define possible column transformations

is_nan = lambda x: x != x
equals = lambda a, b: (is_nan(a) and is_nan(b)) or a == b
all_unique_values = lambda column_name: set(training___data[column_name].unique()) | set(validation_data[column_name].unique())

def drop(column_name, data_set, *_):
    data_set.drop(column_name, axis=1, inplace=True)
    return data_set

def drop_row(column_name, data_set, condition):
    return data_set[~data_set[column_name].apply(condition)]

def one_hot(column_name, data_set, options):
    keys = options if options else data_set[column_name].unique()
    for key in keys:
        data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
    return drop(column_name, data_set)

def re_map(column_name, data_set, mapping):
    data_set[column_name] = data_set[column_name].apply(lambda x: mapping[x])
    return data_set

def MSZoning(column_name, data_set, *_):
    options = dict(
        # A="Agriculture",
        C="Commercial",
        FV="Floating Village Residential",
        # I="Industrial",
        RH="Residential High Density",
        RL="Residential Low Density",
        # RP="Residential Low Density Park",
        RM="Residential Medium Density",
    )
    for key, name in options.items():
        if key == "C":
            continue
        data_set[f"zoned_{name}"] = data_set[column_name].apply(lambda x: 1 if x in [key, "C"] else 0)
    return drop(column_name, data_set)

def masonry(column_name, data_set, *_):
    options = dict(
        BrkCmn="Brick Common",
        BrkFace="Brick Face",
        CBlock="Cinder Block",
        Stone="Stone",
    )
    for key in options:
        data_set[f"masonry {options[key]}"] = 0
        for id in range(len(data_set)):
            if data_set[column_name].iloc[id] == key:
                data_set[f"masonry {options[key]}"].iat[id] = data_set["MasVnrArea"].iloc[id]
    
    data_set.drop(["MasVnrType", "MasVnrArea"], axis=1, inplace=True)
    return data_set

import inflation
def sale_price(column_name, data_set, *_):
    for id in range(len(data_set)):
        data_set[column_name].iat[id] = inflation.adjust(
            price = data_set[column_name].iloc[id],
            year = data_set["YrSold"].iloc[id],
            month = data_set["MoSold"].iloc[id],
        )
    data_set.drop(["MoSold", "YrSold"], axis=1, inplace=True)
    return data_set

In [19]:
# specify each column transformation

transformations = {
    "MSSubClass": (drop, None),
    "MSZoning": (MSZoning, None),
    "LotFrontage": (drop_row, is_nan),
    "Street": (re_map, {"Pave": 1, "Grvl": 0}),
    "Alley": (re_map, {"Pave": 2, "Grvl": 1, nan: 0}),
    "LotShape": (re_map, {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3}),
    "LandContour": (re_map, {"Lvl": 1, "Bnk": 3, "HLS": 2, "Low": 0}),
    "Utilities": (drop, None),
    "LotConfig": (one_hot, all_unique_values("LotConfig")),
    "LandSlope": (re_map, {"Gtl": 0, "Mod": 1, "Sev": 2}),
    "Neighborhood": (one_hot, all_unique_values("Neighborhood")),
    "Condition1": (drop, None),
    "Condition2": (drop, None),
    "BldgType": (one_hot, all_unique_values("BldgType")),
    "YearBuilt": (drop, None),
    "RoofStyle": (one_hot, all_unique_values("RoofStyle")),
    "RoofMatl": (one_hot, all_unique_values("RoofMatl")),
    "Exterior1st": (one_hot, all_unique_values("Exterior1st")),
    "Exterior2nd": (drop, None),
    "MasVnrType": (masonry, None),
    "ExterQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "ExterCond": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Foundation": (one_hot, all_unique_values("Foundation")),
    # "BsmtQual": (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 3}),
    # TODO: handle everything to do with basement
    "TotalBsmtSF": (drop, None),
    "Heating": (one_hot, all_unique_values("Heating")),
    "HeatingQC": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "CentralAir": (re_map, {"Y": 1, "N": 0}),
    "Electrical": (re_map, {"SBrkr": 4, "FuseA": 3, "FuseF": 2, "FuseP": 1, "Mix": 0, nan: 0}),
    "KitchenQual": (re_map, {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0}),
    "Functional": (re_map, {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3, "Maj2": 2, "Sev": 1, "Sal": 0}),
    "FireplaceQu": (re_map, {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, nan: 3}),
    # TODO: handle everything to do with garages
    "PavedDrive": (re_map, {"Y": 2, "P": 1, "N": 0}),
    "PoolQC": (re_map, {"Ex": 5, "Gd": 4, "TA": 2, "Fa": 1, nan: 3}),
    "Fence": (one_hot, all_unique_values("Fence")),
    "MiscFeature": (drop, None),
    "SaleType": (one_hot, all_unique_values("SaleType")),
    "SaleCondition": (one_hot, all_unique_values("SaleCondition")),
    "SalePrice": (sale_price, None),
}

In [20]:
# apply column transformations to data
load_data()
for column_name, (transform, option) in transformations.items():
    print("transforming", column_name)
    training___data = transform(column_name, training___data, option)
    validation_data = transform(column_name, validation_data, option)

validation_data

transforming MSSubClass
transforming MSZoning
transforming LotFrontage
transforming Street
transforming Alley
transforming LotShape
transforming LandContour
transforming Utilities
transforming LotConfig
transforming LandSlope
transforming Neighborhood
transforming Condition1
transforming Condition2
transforming BldgType
transforming YearBuilt
transforming RoofStyle
transforming RoofMatl
transforming Exterior1st
transforming Exterior2nd
transforming MasVnrType
transforming ExterQual
transforming ExterCond
transforming Foundation
transforming TotalBsmtSF
transforming Heating
transforming HeatingQC
transforming CentralAir
transforming Electrical
transforming KitchenQual
transforming Functional
transforming FireplaceQu
transforming PavedDrive
transforming PoolQC
transforming Fence
transforming MiscFeature
transforming SaleType
transforming SaleCondition
transforming SalePrice


/tmp/ipykernel_13585/1595320465.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
/tmp/ipykernel_13585/1595320465.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_set[f"{column_name}_{key}"] = data_set[column_name].apply(lambda x: 1 if equals(x, key) else 0)
/tmp/ipykernel_13585/1595320465.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

,Id,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LandSlope,HouseStyle,OverallQual,...,SaleType_Oth,SaleType_COD,SaleType_Con,SaleType_ConLI,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Normal,SaleCondition_AdjLand,SaleCondition_Family,SaleCondition_Partial
491,492,79.0,9490,1,0,0,1,0,1.5Fin,6,...,0,0,0,0,0,0,1,0,0,0
279,280,83.0,10005,1,0,0,1,0,2Story,7,...,0,0,0,0,0,0,1,0,0,0
655,656,21.0,1680,1,0,0,1,0,2Story,6,...,0,0,0,0,0,0,0,0,1,0
1013,1014,60.0,7200,1,0,0,1,0,1Story,5,...,0,0,0,0,0,0,1,0,0,0
1403,1404,49.0,15256,1,0,1,1,0,1Story,8,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,584,75.0,13500,1,0,0,1,0,2.5Unf,10,...,0,0,0,0,0,0,1,0,0,0
1245,1246,78.0,12090,1,0,0,1,0,SLvl,6,...,0,0,0,0,0,1,0,0,0,0
1390,1391,70.0,9100,1,0,0,1,0,1Story,7,...,0,0,0,0,0,0,1,0,0,0
1375,1376,89.0,10991,1,0,1,2,0,1Story,8,...,0,0,0,0,0,0,0,0,0,1


In [21]:
training___data.describe()
validation_data.describe()

,Id,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LandSlope,OverallQual,OverallCond,...,SaleType_Oth,SaleType_COD,SaleType_Con,SaleType_ConLI,SaleCondition_Alloca,SaleCondition_Abnorml,SaleCondition_Normal,SaleCondition_AdjLand,SaleCondition_Family,SaleCondition_Partial
count,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,...,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.00000,305.000000
mean,710.842623,70.888525,9744.954098,0.996721,0.104918,0.308197,1.137705,0.032787,6.193443,5.639344,...,0.003279,0.029508,0.003279,0.003279,0.013115,0.065574,0.786885,0.003279,0.02623,0.104918
std,422.048717,27.455119,5841.765318,0.057260,0.383211,0.522612,0.480599,0.178371,1.486327,1.121453,...,0.057260,0.169504,0.057260,0.057260,0.113953,0.247942,0.410181,0.057260,0.16008,0.306951
min,2.000000,21.000000,1477.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,363.000000,60.000000,7200.000000,1.000000,0.000000,0.000000,1.000000,0.000000,5.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
50%,668.000000,69.000000,9084.000000,1.000000,0.000000,0.000000,1.000000,0.000000,6.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
75%,1074.000000,80.000000,11310.000000,1.000000,0.000000,1.000000,1.000000,0.000000,7.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
max,1455.000000,313.000000,63887.000000,1.000000,2.000000,3.000000,3.000000,1.000000,10.000000,9.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
